# Version 3

## Test Coverage

1. Train에서 공급유형이 장기전세, 공공분양, 공공임대(5년)인 example을 아예 Drop한다.
2. 각 단지코드 내에 임대건물구분 -> '아파트'이면 1, '상가&아파트'이면 0
3. '자격유형별 평균 임대료' feature 추가

## Import Module

In [1]:
import pandas as pd
import numpy as np
import os
from tqdm.notebook import tqdm

## Data Load

In [2]:
DATA_ROOT = ''
DATA_ROOT = os.path.join(DATA_ROOT, "../../../competition_data/parking_data/")

TRAIN_ROOT = os.path.join(DATA_ROOT, "train.csv")
TEST_ROOT = os.path.join(DATA_ROOT, "test.csv")
AGE_GENDER_INFO_ROOT = os.path.join(DATA_ROOT, "age_gender_info.csv")

print(f"DATA_ROOT : {DATA_ROOT}")
print(f"TRAIN_ROOT : {TRAIN_ROOT}")
print(f"TEST_ROOT : {TEST_ROOT}")
print(F"SUBMISSION_ROOT : {AGE_GENDER_INFO_ROOT}")

DATA_ROOT : ../../../competition_data/parking_data/
TRAIN_ROOT : ../../../competition_data/parking_data/train.csv
TEST_ROOT : ../../../competition_data/parking_data/test.csv
SUBMISSION_ROOT : ../../../competition_data/parking_data/age_gender_info.csv


In [3]:
train = pd.read_csv(TRAIN_ROOT)
test = pd.read_csv(TEST_ROOT)
age_gender_info = pd.read_csv(AGE_GENDER_INFO_ROOT)

print("Data Loaded!")

Data Loaded!


## Preprocessing (Version 2)

### `'-'` -> NULL, dtype을 float으로 변경

In [4]:
columns = ['임대보증금', '임대료']

for col in columns:
    train.loc[train[col] == '-', col] = np.nan
    test.loc[test[col] == '-', col] = np.nan

    train[col] = train[col].astype(float)
    test[col] = test[col].astype(float)

### NULL 값 처리

#### 임대보증금, 임대료

In [5]:
train[['임대보증금', '임대료']] = train[['임대보증금', '임대료']].fillna(0)
test[['임대보증금', '임대료']] = test[['임대보증금', '임대료']].fillna(0)

#### 지하철, 버스

In [6]:
cols = ['도보 10분거리 내 지하철역 수(환승노선 수 반영)', '도보 10분거리 내 버스정류장 수']
train[cols] = train[cols].fillna(0)
test[cols] = test[cols].fillna(0)

#### 자격유형

In [7]:
test.loc[test.단지코드.isin(['C2411']) & test.자격유형.isnull(), '자격유형'] = 'A'
test.loc[test.단지코드.isin(['C2253']) & test.자격유형.isnull(), '자격유형'] = 'C'

#### 중복 example 제거

In [8]:
train = train.drop_duplicates()
test = test.drop_duplicates()

## Preprocessing (Version 3)